<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Trades-Endpoints/blob/main/Place_order_in_demo_trading_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 模擬交易下單

In [ ]:
# POST /openApi/swap/v2/trade/order

# UID 速率限制：5 次/秒
# IP 分組速率限制：

# API KEY 權限需求：永續合約交易

In [ ]:
# 載入時間模組
import time
# 載入requests模組，負責HTTP請求
import requests
# 載入hmac模組，負責簽名
import hmac
# 載入sha256雜湊演算法
from hashlib import sha256

In [ ]:
# 定義API主機
APIURL = "https://open-api.bingx.com"
# 定義API金鑰（需自行填入）
APIKEY = ""
# 定義SECRET金鑰（需自行填入）
SECRETKEY = ""

In [ ]:
# 下單主程式（範例）
def demo():
    # 建立payload資料（POST主體，這裡是空的）
    payload = {}
    # API路徑
    path = '/openApi/swap/v2/trade/order'
    # HTTP請求方法
    method = "POST"
    # 下單參數（dict格式）
    paramsMap = {
        "symbol": "BTC-USDT",  # 交易對
        "side": "BUY",         # 買單
        "positionSide": "LONG",# 多單
        "type": "MARKET",      # 市價單
        "quantity": 5,         # 下單數量
        "takeProfit": "{\"type\": \"TAKE_PROFIT_MARKET\", \"stopPrice\": 31968.0,\"price\": 31968.0,\"workingType\":\"MARK_PRICE\"}"
        # 止盈條件，必須字串格式
    }
    # 將參數格式化並加入timestamp
    paramsStr = parseParam(paramsMap)
    # 實際發送API請求
    return send_request(method, path, paramsStr, payload)

In [ ]:
# HMAC簽名產生函數
def get_sign(api_secret, payload):
    # 計算簽名值
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    # 印出簽名（方便debug）
    print("sign=" + signature)
    # 回傳簽名字串
    return signature

In [ ]:
# 發送HTTP請求主體
def send_request(method, path, urlpa, payload):
    # 組合帶簽名的請求網址
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    # 印出實際請求網址
    print(url)
    # 設定APIKEY於header
    headers = {
        'X-BX-APIKEY': APIKEY,
    }
    # 發送HTTP請求
    response = requests.request(method, url, headers=headers, data=payload)
    # 回傳伺服器回應字串
    return response.text

In [ ]:
# 將參數字典轉為排序後的查詢字串，並加上timestamp
def parseParam(paramsMap):
    # 取得排序後的鍵名
    sortedKeys = sorted(paramsMap)
    # 依序組合成查詢字串
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])
    # 如果有參數，加上&timestamp
    if paramsStr != "":
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        # 沒有其他參數直接加timestamp
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
# 主程式區段（進入點）
if __name__ == '__main__':
    # 執行demo函式並印出回應
    print("demo:", demo())